# Comparaison of regression lines and analysis of covariance

## 1. Introduction

Previously, we discussed simple linear regression which involves fitting a straight line to a number of points using the method of least squares. In this chapter we show how to compare two or more regression lines. We also show how this technique can be used with analysis of variance to control experimental error, a method called analysis of covariance.

## 2. Comparison of two regression lines

Here we describe how to test whether regression lines fitted to two independent sets of data are parallel. Using the two independent estimates of slope $\hat{\beta_1}$, $\hat{\beta_2}$ based on $n_1$ and $n_2$ observations, you wish to test the hypothesis that $\hat{\beta_1} = \hat{\beta_2}$. If you can assume parallel lines you can go on to test whether the intercepts are equal. If they are you can fit an overall line to the data.

In our working example, a greenhouse experiment was carried out to investigate the response of strawberry to two types of fertiliser (F1 and F2). Four plants were grown in each pot and twelve pots were treated at random with each fertiliser. At harvest time various measurements were made on each pot, but for this example we are only interested in the total fruit dry weight (X) and the total leaf area(Y).

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t, f, probplot, linregress, levene

In [3]:
#dataframe of experimental data
data = pd.DataFrame({'fertiliser':[1 for _ in range(12)] + [2 for _ in range(12)],
                     'X':[0.29, 0.43, 0.21, 0.53, 0.27, 0.33, 0.47, 0.40, 0.48, 0.30, 0.37, 0.30,
                          0.27, 0.37, 0.42, 0.19, 0.30, 0.25, 0.35, 0.48, 0.22, 0.30, 0.14, 0.32],
                     'Y':[144, 180, 60, 226, 105, 111, 217, 221, 218, 137, 153, 105,
                          129, 206, 172, 80, 124, 89, 134, 220, 138, 105, 62, 181]})

### 2.1 Testing for equality of slopes

To test the hypothesis that $\hat{\beta_1} = \hat{\beta_2}$ carry out a t-test by calculating $t = (\hat{\beta_1} - \hat{\beta_2}) \: / \: SED$ and calculate a p-value on $n_1 + n_2 - 4$ degrees of freedom where SED is the standard error of the difference between the two fitted slopes.

$$ SED = \sqrt{s_p^2 \left( \frac{1}{S_{xx1}} + \frac{1}{S_{xx2}} \right)} $$

$$ s_p^2 = \frac{ResidSS_1 + ResidSS_2}{(n_1 - 2) + (n_2 - 2)} $$

$s_p^2$ is an estimate of the assumed common population residual variance. When $n_1 = n_2$ this is the average of the two residual mean squares $RMS_1$ and $RMS_2$. To test the assumption of common population variance you divide the larger RMS by the smaller to obtain an F-value and then calculate a p-value on $n - 2$ degrees of freedom.

In [44]:
#get x and y values for the two treatments
X1 = data[data['fertiliser']==1]['X']
X2 = data[data['fertiliser']==2]['X']
Y1 = data[data['fertiliser']==1]['Y']
Y2 = data[data['fertiliser']==2]['Y']

#sample size
n1 = len(data[data['fertiliser']==1])
n2 = len(data[data['fertiliser']==2])

#degrees of freedom
df1 = n1 - 2
df2 = n2 - 2
df_resid = df1 + df2

#get sum of squares
Sxx1 = X1.var() * (n1 - 1)
Sxx2 = X2.var() * (n2 - 1)

#perform  linear regression
slope1, intercept1, rval1, pval1, stderr1 = linregress(X1, Y1)
slope2, intercept2, rval2, pval2, stderr2 = linregress(X2, Y2)

#fitted values
Y1_fit = X1 * slope1 + intercept1
Y2_fit = X2 * slope2 + intercept2

#residual sum of squares
resid_ss1 = np.sum((Y1 - Y1_fit) ** 2)
resid_ss2 = np.sum((Y2 - Y2_fit) ** 2)

#residual mean squares
resid_ms1 = resid_ss1 / df1
resid_ms2 = resid_ss2 / df2

#common population residual variance
sp2 = (resid_ss1 + resid_ss2) / (n1 + n2 - 4)

#standard error of the difference between the slopes
sed = np.sqrt(sp2 * (1 / Sxx1 + 1 / Sxx2))

t_val = (slope1 - slope2) / sed

p_val = 1 - t.cdf(abs(t_val), df_resid)

The p-value for the test that the slopes are equal is 0.22 so we cannot reject this hypothesis. The data are thus in agreement with the hypothesis of a common slope. The estimate for the common slope is:

$$  \hat{\beta} = \frac{S_{xy1} + S_{xy2}}{S_{xx1} + S_{xx2}} $$

In [ ]:
def linear_model(x, y):
    """Perform a linear regression of Y on X"""
    #corrected sum of products Sxy
    Sxy = np.sum(x * y) - np.sum(x) * np.sum(y) / len(x)

    #corrected sum of squares of X
    Sxx = np.sum(x ** 2) - np.sum(x) ** 2 / len(x)

    #calculation of the parameters of the regression line y = a + b * x
    b = Sxy / Sxx
    a = np.mean(y) - b * np.mean(x)
    
    #calculate fitted y values
    y_hat = np.array([a + b * i for i in x])
    
    #calculation of residuals
    resid = np.zeros(len(y))
    for i in range(len(y)):
        resid[i] = y[i] - y_hat[i]

    print("Verification of the assumptions of the linear regression\n")
    #plotting of residuals
    fig, ((ax0, ax1), (ax2, _)) = plt.subplots(nrows=2, ncols=2)
    fig.delaxes(_)
    plt.subplots_adjust(wspace=0.5, hspace=0.8)
    ax0.hist(resid)
    ax0.set_xlabel("Residuals")
    ax0.set_ylabel("Frequency")
    ax0.set_title("Histogram of residuals")
    probplot(resid, plot=ax1)
    ax1.set_xlabel("Normal score")
    ax1.set_ylabel("Residuals")
    ax1.set_title("Q-Q plot of residuals")
    ax2.scatter(y_hat, resid)
    ax2.set_xlabel("Fitted values")
    ax2.set_ylabel("Residuals")
    ax2.set_title("Residuals VS fitted values")
    ax2.axhline(y=0, linewidth=0.5, color="grey")
    plt.show()
    
    #calculation of R-sq
    SSE = np.sum((y - y_hat) ** 2)
    SSR = np.sum((y_hat - np.mean(y)) ** 2)
    Rsq = SSR / (SSE + SSR)
    
    #calculation of adjusted R-sq
    #total mean square
    tms = np.sum((y - np.mean(y)) ** 2) / (len(y) - 1)
    Rsq_adj = (tms - SSE / (len(y) - 2)) / tms
    
    #testing null hypothesis that b = 0
    #degrees of freedom of MSE
    dfd = len(y) - 2
    MSE = SSE / dfd
    #calculation of the standard error of b
    SE_b = sqrt(MSE / Sxx)
    #calculation of t-value
    t_val_b =  b / SE_b
    #calculation of p-value
    p_val_b = 2 * (1 - t.cdf(abs(t_val_b), df=dfd))
    
    #testing the significance of the regression (analysis of variance)
    #degrees of freedom of MSR
    dfn = 1
    #variance ratio
    MSR = SSR / 1
    vr = MSR / MSE
    #p-value
    p_val_f = 1 - f.cdf(vr, dfn, dfd)
    
    #test whether intercept is significantly different from zero
    SE_a = sqrt(MSE * (1 / len(y) + np.mean(x) ** 2 / Sxx))
    t_val_a = a / SE_a
    p_val_a = 2 * (1 - t.cdf(abs(t_val_a), df=dfd))
    
    #calculate confidence interval values for fitted Y values
    x_val = np.linspace(min(x), max(x), 200)
    y_fit = np.array([a + b * i for i in x_val])
    ci_lo = np.zeros(len(x_val))
    ci_hi = np.zeros(len(x_val))
    for i in range(len(x_val)):
        ci_lo[i] = y_fit[i] + t.ppf(0.025, df=len(y) - 2) * sqrt(MSE * (1 / len(y) + (x_val[i] - np.mean(x)) ** 2 / Sxx))
        ci_hi[i] = y_fit[i] + t.ppf(0.975, df=len(y) - 2) * sqrt(MSE * (1 / len(y) + (x_val[i] - np.mean(x)) ** 2 / Sxx))

    #plot linear regression with confidence interval on fitted Y values
    fig, ax = plt.subplots()
    ax.scatter(x, y, color="C0", label="experimental points")
    ax.plot(x, y_hat, color="C3", label="regression line")
    ax.plot(x_val, ci_lo, color="black", linestyle="--", linewidth=0.5, label="95% CI")
    ax.plot(x_val, ci_hi, color="black", linestyle="--", linewidth=0.5)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("Linear regression of y on x")
    #ax.set_xlim(-1, 101)
    #ax.set_ylim(-1, 22)
    ax.legend()
    plt.show()
    
    print("The regression equation is")
    print("Y = {0:.3f} {1:+.3f} * X\n".format(a, b))
    print("{0:11}{1:>10}{2:>10}{3:>10}{4:>10}".format("", "Coef", "StDev", "t-value", "p-value"))
    print("{0:11}{1:10.4f}{2:10.4f}{3:10.4f}{4:10.5f}".format("Intercept", a, SE_a, t_val_a, p_val_a))
    print("{0:11}{1:10.4f}{2:10.4f}{3:10.4f}{4:10.5f}\n".format("Slope", b, SE_b, t_val_b, p_val_b))
    print("R-sq = {0:.1f}%, R-sq(adj) = {1:.1f}%\n".format(Rsq*100, Rsq_adj*100))
    print("Analysis of variance")
    print("{0:15}{1:>10}{2:>10}{3:>10}{4:>10}{5:>10}".format("Source", "df", "Sum sq", "Mean sq", "F-value", "p-value"))
    print("{0:15}{1:10}{2:10.2f}{3:10.2f}{4:10.3f}{5:10.5f}".format("Regression", 1, SSR, MSR, vr, p_val_f))
    print("{0:15}{1:10}{2:10.2f}{3:10.2f}".format("Residual error", dfd, SSE, MSE))
    print("{0:15}{1:10}{2:10.2f}\n".format("Total", dfd + 1, SSR + SSE))